## Session 6


Convolutional Autoencoder

In [ ]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn
from model import Encoder, Decoder
from evaluator import ModelEvaluator, AutoEncoderEvaluator


cudnn.benchmark = True

# Hyperparameters and DataLoader

In [ ]:
cudnn.benchmark = True


trainset = dsets.CIFAR10('./data', train=True, download=True, transform=transforms.ToTensor())
testset = dsets.CIFAR10('./data', train=False, download=True, transform=transforms.ToTensor())
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')



# Set Hyperparameters
epochs = 10
batch_size = 50
lr = 0.001

# Data Loader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=3)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=3)


# Train CAE Without Noise

In [ ]:
# Model
l2 = 0.0
optim = 'adam'
# Pytorch Cross Entropy Loss
encoder = Encoder(batch_size=batch_size)
decoder = Decoder(batch_size=batch_size)
add_noise = False
AEeval = AutoEncoderEvaluator(encoder, decoder, epochs, lr, batch_size=batch_size, l2=l2, add_noise=add_noise, use_gpu=True, optim=optim)
AEeval.evaluator(trainloader, testloader, print_every=100)

modelname = 'model_learning_rate_{}_optimizer_{}_'.format(lr, opt)
torch.save([modeleval.encoder.state_dict(), modeleval.decoder.state_dict()], modelname)

In [ ]:
# Plot training and test loss
AEeval.plot_loss()

## Use latent features to train and test a simple MLP

In [ ]:
n_in = 32768 #512*8*8
n_hidden = 512
n_out = 10
model_epochs = 20
model_lr = 0.001
l2 = 0.0

#train classifier
encoder = AEeval.encoder
model = LogisticRegression(n_in, n_hidden, n_out)
modeleval = ModelEvaluator(model, model_epochs, model_lr, l2=l2, use_gpu=True, optim='adam')
acc_ = modeleval.evaluator(encoder, trainloader, testloader, noise=add_noise, print_every=100, validation=False)

print('Accuracy on test set {.2f}'.format(acc_))

In [ ]:
modeleval.plot_loss()

## Denoising AutoEncoder

Add noise to encoder

In [ ]:
# Model
l2 = 0.0
optim = 'adam'
# Pytorch Cross Entropy Loss
encoder = Encoder(batch_size=batch_size)
decoder = Decoder(batch_size=batch_size)
add_noise = True
AEeval1 = AutoEncoderEvaluator(encoder, decoder, epochs, lr, batch_size=batch_size, l2=l2, add_noise=add_noise, use_gpu=True, optim=optim)
AEeval1.evaluator(trainloader, testloader, print_every=100)

modelname = 'model_denoise_learning_rate_{}_optimizer_{}_'.format(lr, opt)
torch.save([modeleval.encoder.state_dict(), modeleval.decoder.state_dict()], modelname)

## Use latent features to train and test a simple MLP

In [ ]:
n_in = 32768 #512*8*8
n_hidden = 512
n_out = 10
model_epochs = 20
model_lr = 0.01
l2 = 0.0
#train classifier
encoder = AEeval.encoder
model = LogisticRegression(n_in, n_hidden, n_out)
modeleval = ModelEvaluator(model, model_epochs, model_lr, l2=l2, use_gpu=True, optim='adam')
acc_ = modeleval.evaluator(encoder, trainloader, testloader, noise=add_noise, print_every=100, validation=False)

print('Accuracy on test set {.2f}'.format(acc_))

In [ ]:
modeleval.plot_loss()

## Get latent features from Soccer Dataset and train a logistic regression classifier

In [ ]:
n_in = 32768 #512*8*8
n_hidden = 512
n_out = 2
batch_size = 100
epochs = 20
noise = False
l2 = 0.0
lr = 0.01
model = LogisticRegression(n_in, n_hidden, n_out)
modeleval = ModelEvaluator(model, epochs, lr, l2, use_gpu=True, optim='adam')

In [ ]:
transform = transforms.Compose([
        transforms.RandomResizedCrop(32),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    
dataset = SoccerDataset(transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=3)

In [ ]:
for epoch in range(epochs):
    model.train()
    modeleval.train(encoder, epoch, dataloader, noise=False, print_every=100)

## Test classifier on new images

In [ ]:
test_path = 'Session6/SoccerData/test'

def test_img(path, encoder, model):
    img = Image.open(path)
    plt.imshow(img)
    transform = transforms.Compose([
        transforms.Resize(64),
        transforms.CenterCrop(32),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    img = transform(img)
    img = img.numpy()
    img = img[np.newaxis, ...]
    img = torch.Tensor(img)

    model.eval()

    with torch.no_grad():
        latent_repr = encoder.forward(img)
        output = model.forward(latent_repr)
        _, idx = torch.max(test_preds.data, 1)
        print(str(output))
        print('prediction: %s' % ['not soccer', 'soccer'][idx])

In [ ]:
img1_path = test_path + '/test1.jpg'
test_img(img1_path, encoder, model)